In [1]:
# Import the Pandas package
import pandas as pd
import numpy as np
import re 
import warnings
warnings.filterwarnings("ignore")  # Suppress all warnings

In [6]:
def clean_text(string,replace='_'):
    regex = '[^A-Za-z0-9]+'
    str1 = re.sub(regex, replace, string)
    #regex = '[^A-Za-z_]+'
    #return re.sub(regex, '', str1)
    return str1

def load_files(year):
    temp_df = pd.read_csv(file_str.format(year), skiprows = 1)
    column_list = list(temp_df.columns)#.to_list()
    if('Population' in column_list or 'population' in column_list):
        return temp_df.rename(columns = lambda col: clean_text(col.lower().strip()))
    else: 
        return pd.read_csv(file_str.format(year), skiprows = 1).rename(columns = lambda col: clean_text(col.lower().strip()))
    
    
def remove_digits(string):
    return re.sub('\d', '', string)


def replace_emdash(string):
    return string.replace('–','-',100)

def replace_all_special(string):
    return re.sub('[^A-Za-z0-9, ]+', '-', string)

def replace_msa(string):
    string= string.replace('Micro Area','')
    string= string.replace('Metro Area','')
    #string= string.replace('M.S.A','')
    return string.strip()

def split_name(string):
    split = string.split(',')
    city= split[0].strip()
    state = split[1].strip()
    
    city_split=city.split('-')
    state_split=state.split('-')
    return city_split[0]+', '+state_split[0]

def clean_string(col):
    return col.apply(remove_digits).apply(replace_emdash).apply(replace_msa).apply(replace_all_special).apply(split_name)#.apply(replace_all_special)


In [71]:
file_str = "data/demographics/poverty_census_{}.csv"
dfs = []
for  i in range(2010,2018):
    dfs.append(load_files(i))

In [72]:
for i in dfs:
    print(i.shape)

(525, 278)
(529, 278)
(530, 278)
(515, 278)
(515, 278)
(516, 368)
(518, 368)
(518, 368)


In [73]:
columnstokeep = [
    'geographic_area_name',
    'total_estimate_sex_male',
    'total_estimate_sex_female',
    'total_estimate_race_and_hispanic_or_latino_origin_one_race',
    'total_estimate_race_and_hispanic_or_latino_origin_one_race_white',
    'total_estimate_race_and_hispanic_or_latino_origin_one_race_black_or_african_american',
    'total_estimate_race_and_hispanic_or_latino_origin_one_race_american_indian_and_alaska_native',
    'total_estimate_race_and_hispanic_or_latino_origin_one_race_asian',
    'total_estimate_race_and_hispanic_or_latino_origin_one_race_native_hawaiian_and_other_pacific_islander',
    'total_estimate_race_and_hispanic_or_latino_origin_one_race_some_other_race',
    'total_estimate_race_and_hispanic_or_latino_origin_two_or_more_races',
    'total_estimate_hispanic_or_latino_origin_of_any_race_',
    'total_estimate_white_alone_not_hispanic_or_latino',
    
    'below_poverty_level_estimate_population_for_whom_poverty_status_is_determined',
    'below_poverty_level_estimate_sex_male',
    'below_poverty_level_estimate_sex_female',
    'below_poverty_level_estimate_employment_status_employed',
    'below_poverty_level_estimate_employment_status_employed_male',
    'below_poverty_level_estimate_employment_status_employed_female',
    'below_poverty_level_estimate_employment_status_unemployed',
    'below_poverty_level_estimate_employment_status_unemployed_male',
    'below_poverty_level_estimate_employment_status_unemployed_female',
    
    'below_poverty_level_estimate_race_and_hispanic_or_latino_origin_one_race',
    'below_poverty_level_estimate_race_and_hispanic_or_latino_origin_one_race_white',
    'below_poverty_level_estimate_race_and_hispanic_or_latino_origin_one_race_black_or_african_american',
    'below_poverty_level_estimate_race_and_hispanic_or_latino_origin_one_race_american_indian_and_alaska_native',
    'below_poverty_level_estimate_race_and_hispanic_or_latino_origin_one_race_asian',
    'below_poverty_level_estimate_race_and_hispanic_or_latino_origin_one_race_native_hawaiian_and_other_pacific_islander',
    'below_poverty_level_estimate_race_and_hispanic_or_latino_origin_one_race_some_other_race',
    
    'below_poverty_level_estimate_race_and_hispanic_or_latino_origin_two_or_more_races',
    'below_poverty_level_estimate_hispanic_or_latino_origin_of_any_race_',
    'below_poverty_level_estimate_white_alone_not_hispanic_or_latino',
    'below_poverty_level_estimate_educational_attainment_population_25_years_and_over',
    'below_poverty_level_estimate_educational_attainment_less_than_high_school_graduate',
    'below_poverty_level_estimate_educational_attainment_high_school_graduate_includes_equivalency_',
    'below_poverty_level_estimate_educational_attainment_some_college_associate_s_degree',
    'below_poverty_level_estimate_educational_attainment_bachelor_s_degree_or_higher',
    
    'total_estimate_employment_status_employed',
    'total_estimate_employment_status_employed',
    'total_estimate_employment_status_employed_male',
    'total_estimate_employment_status_employed_female',
    'total_estimate_employment_status_unemployed',
    'total_estimate_employment_status_unemployed_male',
    'total_estimate_employment_status_unemployed_female',
    'percent_below_poverty_level_estimate_did_not_work',
    'total_estimate_did_not_work'
    
]

In [74]:
for d in dfs:
    try:
        d[columnstokeep]
    except:
        print(d.shape)
        for j in columnstokeep: 
            if j not in d.columns:
                print(j)
        print('-------------------')
    

(515, 278)
below_poverty_level_estimate_employment_status_employed
below_poverty_level_estimate_employment_status_employed_male
below_poverty_level_estimate_employment_status_employed_female
below_poverty_level_estimate_employment_status_unemployed
below_poverty_level_estimate_employment_status_unemployed_male
below_poverty_level_estimate_employment_status_unemployed_female
below_poverty_level_estimate_educational_attainment_less_than_high_school_graduate
below_poverty_level_estimate_educational_attainment_high_school_graduate_includes_equivalency_
below_poverty_level_estimate_educational_attainment_some_college_associate_s_degree
below_poverty_level_estimate_educational_attainment_bachelor_s_degree_or_higher
total_estimate_employment_status_employed
total_estimate_employment_status_employed
total_estimate_employment_status_employed_male
total_estimate_employment_status_employed_female
total_estimate_employment_status_unemployed
total_estimate_employment_status_unemployed_male
total_es

In [60]:
for j in columnstokeep: 
    if j in dfs[-1].columns:
        print(j)

geographic_area_name
total_estimate_sex_male
total_estimate_sex_female
total_estimate_race_and_hispanic_or_latino_origin_two_or_more_races
total_estimate_hispanic_or_latino_origin_of_any_race_
total_estimate_white_alone_not_hispanic_or_latino
below_poverty_level_estimate_population_for_whom_poverty_status_is_determined
below_poverty_level_estimate_sex_male
below_poverty_level_estimate_sex_female
below_poverty_level_estimate_race_and_hispanic_or_latino_origin_two_or_more_races
below_poverty_level_estimate_hispanic_or_latino_origin_of_any_race_
below_poverty_level_estimate_white_alone_not_hispanic_or_latino
below_poverty_level_estimate_educational_attainment_population_25_years_and_over
percent_below_poverty_level_estimate_did_not_work
total_estimate_did_not_work


In [69]:
for i in dfs[0].columns[~dfs[0].columns.str.contains('margin')]:
    print(i)

id
geographic_area_name
total_estimate_population_for_whom_poverty_status_is_determined
total_estimate_age_under_years
total_estimate_age_under_years_related_children_under_years
total_estimate_age_to_years
total_estimate_age_years_and_over
total_estimate_sex_male
total_estimate_sex_female
total_estimate_race_and_hispanic_or_latino_origin_one_race
total_estimate_race_and_hispanic_or_latino_origin_one_race_white
total_estimate_race_and_hispanic_or_latino_origin_one_race_black_or_african_american
total_estimate_race_and_hispanic_or_latino_origin_one_race_american_indian_and_alaska_native
total_estimate_race_and_hispanic_or_latino_origin_one_race_asian
total_estimate_race_and_hispanic_or_latino_origin_one_race_native_hawaiian_and_other_pacific_islander
total_estimate_race_and_hispanic_or_latino_origin_one_race_some_other_race
total_estimate_race_and_hispanic_or_latino_origin_two_or_more_races
total_estimate_hispanic_or_latino_origin_of_any_race_
total_estimate_white_alone_not_hispanic_or_

In [68]:
for i in dfs[-2].columns[~dfs[-2].columns.str.contains('margin')]:
    print(i)

id
geographic_area_name
total_estimate_population_for_whom_poverty_status_is_determined
total_estimate_age_under_years
total_estimate_age_under_years_under_years
total_estimate_age_under_years_to_years
total_estimate_age_under_years_related_children_of_householder_under_years
total_estimate_age_to_years
total_estimate_age_to_years_to_years
total_estimate_age_to_years_to_years
total_estimate_age_years_and_over
total_estimate_age_years_and_over
total_estimate_sex_male
total_estimate_sex_female
total_estimate_race_and_hispanic_or_latino_origin_white_alone
total_estimate_race_and_hispanic_or_latino_origin_black_or_african_american_alone
total_estimate_race_and_hispanic_or_latino_origin_american_indian_and_alaska_native_alone
total_estimate_race_and_hispanic_or_latino_origin_asian_alone
total_estimate_race_and_hispanic_or_latino_origin_native_hawaiian_and_other_pacific_islander_alone
total_estimate_race_and_hispanic_or_latino_origin_some_other_race_alone
total_estimate_race_and_hispanic_or_

In [5]:
for i in range(len(dfs)):
    temp_df = dfs[i]
    temp_df = temp_df[temp_df['geographic_area_name'].str.contains("Area")]
    temp_df['geographic_area_name'] = clean_string(temp_df['geographic_area_name'])
    dfs[i] = temp_df

In [13]:
dfs[1].columns[dfs[1].columns.str.contains('year')]

Index(['total_estimate_age_under__years',
       'total_margin_of_error_age_under__years',
       'total_estimate_age_under__years_related_children_under__years',
       'total_margin_of_error_age_under__years_related_children_under__years',
       'total_estimate_age__to__years', 'total_margin_of_error_age__to__years',
       'total_estimate_age__years_and_over',
       'total_margin_of_error_age__years_and_over',
       'total_estimate_educational_attainment_population__years_and_over',
       'total_margin_of_error_educational_attainment_population__years_and_over',
       'total_estimate_employment_status_civilian_labor_force__years_and_over',
       'total_margin_of_error_employment_status_civilian_labor_force__years_and_over',
       'total_estimate_work_experience_population__years_and_over',
       'total_margin_of_error_work_experience_population__years_and_over',
       'total_estimate_work_experience_worked_full_time_year_round_in_the_past__months',
       'total_margin_of_e

## Race Only Data

In [18]:
file_str = "data/demographics/Race{}.csv"
racedfs = []
for  i in range(2010,2018):
    racedfs.append(load_files(i))

In [24]:
races = [    
    "estimate_sex_and_age_total_population",
    "estimate_sex_and_age_male",
    "percent_sex_and_age_male",
    "estimate_sex_and_age_female",
    "percent_sex_and_age_female",
    "estimate_race_one_race_white",
    "percent_race_one_race_white",
    "estimate_race_one_race_black_or_african_american",
    "percent_race_one_race_black_or_african_american",
    "estimate_hispanic_or_latino_and_race_total_population",
    "percent_hispanic_or_latino_and_race_total_population",
    "estimate_race_native_hawaiian_and_other_pacific_islander",
    "percent_race_native_hawaiian_and_other_pacific_islander"
]

In [19]:
for i in racedfs[0].columns[~racedfs[0].columns.str.contains('margin')]:
    print(i)

id
geographic_area_name
estimate_sex_and_age_total_population
percent_sex_and_age_total_population
estimate_sex_and_age_male
percent_sex_and_age_male
estimate_sex_and_age_female
percent_sex_and_age_female
estimate_sex_and_age_under_5_years
percent_sex_and_age_under_5_years
estimate_sex_and_age_5_to_9_years
percent_sex_and_age_5_to_9_years
estimate_sex_and_age_10_to_14_years
percent_sex_and_age_10_to_14_years
estimate_sex_and_age_15_to_19_years
percent_sex_and_age_15_to_19_years
estimate_sex_and_age_20_to_24_years
percent_sex_and_age_20_to_24_years
estimate_sex_and_age_25_to_34_years
percent_sex_and_age_25_to_34_years
estimate_sex_and_age_35_to_44_years
percent_sex_and_age_35_to_44_years
estimate_sex_and_age_45_to_54_years
percent_sex_and_age_45_to_54_years
estimate_sex_and_age_55_to_59_years
percent_sex_and_age_55_to_59_years
estimate_sex_and_age_60_to_64_years
percent_sex_and_age_60_to_64_years
estimate_sex_and_age_65_to_74_years
percent_sex_and_age_65_to_74_years
estimate_sex_and_age

In [32]:
for i in racedfs[-1].columns[~racedfs[-1].columns.str.contains('margin')]:
    print(i)

id
geographic_area_name
percent_race_two_or_more_races_black_or_african_american_and_american_indian_and_alaska_native
estimate_race_race_alone_or_in_combination_with_one_or_more_other_races_total_population
percent_race_race_alone_or_in_combination_with_one_or_more_other_races_total_population
estimate_race_race_alone_or_in_combination_with_one_or_more_other_races_total_population_white
percent_race_race_alone_or_in_combination_with_one_or_more_other_races_total_population_white
estimate_race_race_alone_or_in_combination_with_one_or_more_other_races_total_population_black_or_african_american
percent_race_race_alone_or_in_combination_with_one_or_more_other_races_total_population_black_or_african_american
estimate_race_race_alone_or_in_combination_with_one_or_more_other_races_total_population_american_indian_and_alaska_native
percent_race_race_alone_or_in_combination_with_one_or_more_other_races_total_population_american_indian_and_alaska_native
estimate_race_race_alone_or_in_combinatio

In [21]:
for i in range(len(racedfs)):
    temp_df = racedfs[i]
    temp_df = temp_df[temp_df['geographic_area_name'].str.contains("Area")]
    temp_df['geographic_area_name'] = clean_string(temp_df['geographic_area_name'])
    racedfs[i] = temp_df

In [31]:
racedfs[3].head()

,id,geographic_area_name,percent_race_one_race_american_indian_and_alaska_native_cherokee_tribal_grouping,percent_margin_of_error_race_one_race_american_indian_and_alaska_native_cherokee_tribal_grouping,estimate_race_one_race_american_indian_and_alaska_native_chippewa_tribal_grouping,margin_of_error_race_one_race_american_indian_and_alaska_native_chippewa_tribal_grouping,percent_race_one_race_american_indian_and_alaska_native_chippewa_tribal_grouping,percent_margin_of_error_race_one_race_american_indian_and_alaska_native_chippewa_tribal_grouping,estimate_race_one_race_american_indian_and_alaska_native_navajo_tribal_grouping,margin_of_error_race_one_race_american_indian_and_alaska_native_navajo_tribal_grouping,...,estimate_race_one_race_black_or_african_american,margin_of_error_race_one_race_black_or_african_american,percent_race_one_race_black_or_african_american,percent_margin_of_error_race_one_race_black_or_african_american,estimate_race_one_race_american_indian_and_alaska_native,margin_of_error_race_one_race_american_indian_and_alaska_native,percent_race_one_race_american_indian_and_alaska_native,percent_margin_of_error_race_one_race_american_indian_and_alaska_native,estimate_race_one_race_american_indian_and_alaska_native_cherokee_tribal_grouping,margin_of_error_race_one_race_american_indian_and_alaska_native_cherokee_tribal_grouping
0,310M200US10140,"Aberdeen, WA",N,N,N,N,N,N,N,N,...,1371,556,1.9,0.8,3174,860,4.5,1.2,N,N
1,310M200US10180,"Abilene, TX",N,N,N,N,N,N,N,N,...,13160,1102,7.8,0.7,575,369,0.3,0.2,N,N
2,310M200US10300,"Adrian, MI",N,N,N,N,N,N,N,N,...,3438,961,3.5,1.0,255,255,0.3,0.3,N,N
3,310M200US10380,"Aguadilla, PR",N,N,N,N,N,N,N,N,...,10774,2380,3.3,0.7,440,451,0.1,0.1,N,N
4,310M200US10420,"Akron, OH",N,N,N,N,N,N,N,N,...,85143,2961,12.1,0.4,1232,685,0.2,0.1,N,N


In [25]:
for i in racedfs:
    print(i.shape)

(525, 326)
(529, 326)
(530, 326)
(515, 326)
(515, 326)
(516, 338)
(518, 338)
(518, 358)


In [30]:
for j in races: 
    if j not in dfs[3].columns:
        print(j)

estimate_sex_and_age_male
percent_sex_and_age_male
estimate_sex_and_age_female
percent_sex_and_age_female
estimate_race_native_hawaiian_and_other_pacific_islander
percent_race_native_hawaiian_and_other_pacific_islander
